## Kütüphaneleri Yükleme

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import pandas as pd

from tqdm import tqdm

## Veriyi Okuma

In [2]:
data = pd.read_excel("../Data/Anxiety_Detection_Data/total_df.xlsx")

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19144 entries, 0 to 19143
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    19143 non-null  object
 1   labels  19144 non-null  object
 2   source  19144 non-null  object
dtypes: object(3)
memory usage: 448.8+ KB


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19144 entries, 0 to 19143
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    19143 non-null  object
 1   labels  19144 non-null  object
 2   source  19144 non-null  object
dtypes: object(3)
memory usage: 448.8+ KB


In [6]:
data.head()

,text,labels,source
0,"Sıkışmış hissetmek (yerine tekrar giriş yok, yiyecek ve içecek yok)",agoraphobia,Reddit
1,"Yakın zamanda başka bir şehre taşındım ve neler olduğunu bilmiyorum, ama etrafımdaki herkes hakkımda bok konuşuyor, komşularım benden nefret ediyor ve beni rahatsız etmek için beni aşağılamaya veya yere vurmaya devam ediyor, kasabamdaki insanlar benimle ilgili dedikodu yapıyor, artık arkadaşlarımla dışarı çıkamıyorum çünkü herkes benden nefret ediyor, ne yapacağımı bilmiyorum ve bunun neden olduğunu anlamıyorum, her gün panik atak geçiriyorum",agoraphobia,Reddit
2,"Panik atak geçirmenin eşiğindeydim, sadece bunu düşünüyordum.",agoraphobia,Reddit
3,Geçen hafta maruz kalma terapimi yaparken sokakta panik atak geçirdim. İyileşmek için merdivende birkaç basamak düşürülmüş gibi hissediyorum. Bir zamanlar önemsiz görevlerin tekrar panik tepkisine neden olduğu bu tür aksilikler yaşayan başka biri var mı?,agoraphobia,Reddit
4,Bazen özgüven ve benlik imajıyla çok mücadele ediyorum.,agoraphobia,Reddit


from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-tc-big-en-tr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Çeviri fonksiyonunu tanımla
def translate_text(text):
    inputs = tokenizer([text], return_tensors="pt", padding=True)
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

# DataFrame'deki her satır için çeviri işlemi uygula ve yeni bir kolona ekle
data['Translated_Text'] = data['Text'].apply(translate_text)

# Sonuçları göster
print(data)


In [7]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-tc-big-en-tr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

C:\Users\halilibrahim.hatun\AppData\Local\miniconda3\envs\psynexa_torch_cpu\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(57060, 1024, padding_idx=57059)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(57060, 1024, padding_idx=57059)
      (embed_positions): MarianSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1

In [12]:
def batch_translate_text(texts, batch_size=4):
    translated_texts = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Translating Process"):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True).to(device)
        translated = model.generate(**inputs)
        translated_batch = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        translated_texts.extend(translated_batch)
        del inputs, translated  # Free up memory
        torch.cuda.empty_cache()
    return translated_texts

texts = data['text'].tolist()
translated_texts = batch_translate_text(texts)

data['Translated_Text'] = translated_texts

Translating Process:   1%|▍                                                               | 33/4786 [01:57<4:41:54,  3.56s/it]


KeyboardInterrupt: 

In [10]:
data.to_excel("../Data/dysthymia_to_tr.xlsx", index=False)

In [ ]:
data.head()

data.to_csv("agoraphobia_translated_to_tr", index=False)

df = pd.read_csv("agoraphobia_translated_to_tr")